In [1]:
### obtain/download files programatically
# import synapseclient
 
# syn = synapseclient.Synapse()
 
# syn.login(email='didier.barradasbautista@kaust.edu.sa',apiKey='DuhA56Ya6ZmxSA70I0s5VwXCX4wxx+HcW9bsnHkGJ6ah4C8o3zuqbHHnB2w6nBO4wsceB2Irdsk5pX/Wsm1w+g==')
#  # Obtain a pointer and download the data
# syn20613568 = syn.get(entity='syn20613568')
 
# syn20613580 = syn.get(entity='syn20613580')

In [57]:
import os, sys 
# from dask_ml.model_selection import train_test_split, RandomizedSearchCV
import dask.dataframe as dd
from dask_ml.xgboost import XGBRegressor
import joblib
from dask.diagnostics import ProgressBar
# from dask_ml.preprocessing import StandardScaler
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold , cross_val_predict , cross_val_score ,train_test_split
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score,classification_report, confusion_matrix,accuracy_score,matthews_corrcoef
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from dask.distributed import Client

In [3]:
client = Client()

In [4]:
# dtype = {'treatment': pd.api.types.CategoricalDtype(['EGF' , 'full' , 'iEGFR' , 'iMEK' , 'iPI3K' , 'iPKC'])}

In [5]:
df= pd.DataFrame()
for x in os.listdir("data"):
    print (x)
    df_temp = dd.read_csv("data/%s"%(x) )  #,dtype=dtype)
    df = dd.concat([df,df_temp],axis=0)

184B5.csv
BT483.csv
HCC1428.csv
HCC1806.csv
HCC202.csv
Hs578T.csv
MCF12A.csv
MDAMB231.csv
MDAMB468.csv
SKBR3.csv
UACC3199.csv
ZR751.csv


In [9]:
df = df.categorize(columns=["time","treatment"])

In [7]:
df = df.repartition(npartitions=200)

In [10]:
my_dummies_treatment = dd.get_dummies(df["treatment"])

In [11]:
# https://stackoverflow.com/questions/46911220/simple-way-to-dask-concatenate-horizontal-axis-1-columns
#my_dummies_treatment= my_dummies_treatment.repartition(npartitions=200)

In [12]:
my_dummies_time = dd.get_dummies(df["time"])

In [28]:
my_dummies_time= my_dummies_time.repartition(npartitions=200)
my_dummies_time.reset_index(drop=True)

,0.0,5.5,7.0,9.0,13.0,17.0,23.0,30.0,40.0,60.0,12.0,16.0,24.0
npartitions=200,,,,,,,,,,,,,
,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8
,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...


In [14]:
print ("filling NA")
# y = y.compute()
with ProgressBar():
    for m in ["p.HER2","p.PLCg2"] :
#         print (df[m])
        
        df["%s_c"%(m)] =df[m].fillna(df[m].mean() )#, inplace=True )


filling NA


In [15]:
df = df.drop(['p.HER2','p.PLCg2','cell_line','time','treatment','cellID','fileID'],axis=1)

In [16]:
X = df[['CyclinB',
 'GAPDH',
 'IdU',
 'Ki.67',
 'p.4EBP1',
 'p.Akt.Ser473.',
 'p.AKT.Thr308.',
 'p.AMPK',
 'p.BTK',
 'p.CREB',
 'p.ERK',
 'p.FAK',
 'p.GSK3b',
 'p.H3',
 'p.HER2_c',
 'p.JNK',
 'p.MAP2K3',
 'p.MAPKAPK2',
 'p.MEK',
 'p.MKK3.MKK6',
 'p.MKK4',
 'p.NFkB',
 'p.p38',
 'p.p53',
 'p.p90RSK',
 'p.PDPK1',
 'p.PLCg2_c',
 'p.RB',
 'p.S6',
 'p.S6K',
 'p.SMAD23',
 'p.SRC',
 'p.STAT1',
 'p.STAT3',
 'p.STAT5']]

In [17]:
y = df[ ['b.CATENIN', 'cleavedCas'] ]

In [39]:
with ProgressBar():
    scaler = StandardScaler()
    scaler.fit(X)
    scaled_data = scaler.transform(X)

In [40]:
# type(scaled_data)

In [41]:
scaled_data = pd.DataFrame(scaled_data,columns=X.columns)  #,index=my_dummies_time.index)

In [42]:
scaled_data.index

RangeIndex(start=0, stop=3545428, step=1)

In [32]:
with ProgressBar():
#     scaled_data = scaled_data.compute()
    my_dummies_treatment = my_dummies_treatment.compute()
    my_dummies_time = my_dummies_time.compute()

In [33]:
# ddf = dd.from_pandas(scaled_data,npartitions=200 )

In [43]:
# my_dummies_time.divisions
print (type(my_dummies_treatment))
print (type(my_dummies_time))

<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [46]:
my_dummies_treatment.reset_index(drop=True,inplace=True)
my_dummies_time.reset_index(drop=True,inplace=True)

In [47]:
# my_dummies_time.tail()

time,0.0,5.5,7.0,9.0,13.0,17.0,23.0,30.0,40.0,60.0,12.0,16.0,24.0
3545423,0,0,0,0,0,0,0,0,0,1,0,0,0
3545424,0,0,0,0,0,0,0,0,0,1,0,0,0
3545425,0,0,0,0,0,0,0,0,0,1,0,0,0
3545426,0,0,0,0,0,0,0,0,0,1,0,0,0
3545427,0,0,0,0,0,0,0,0,0,1,0,0,0


In [ ]:
# for c in my_dummies_time.index.values.tolist():
#     if c == 150000:
#         print (c)

In [48]:
len(scaled_data)

3545428

In [49]:
scaled_data.index

RangeIndex(start=0, stop=3545428, step=1)

In [23]:
# df.npartitions

200

In [51]:
with ProgressBar():
    scaled_data = pd.concat([scaled_data,my_dummies_treatment,my_dummies_time],axis=1)

In [ ]:
# df = df.repartition(npartitions=200)

In [52]:
# genes= ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.', 'p.AKT.Thr308.','p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK','p.MAP2K3','p.MAPKAPK2','p.MEK','p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK','p.PDPK1','p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']

In [56]:
y = y.compute()

In [53]:
rounds = {}
genes= ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.', 'p.AKT.Thr308.','p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK','p.MAP2K3','p.MAPKAPK2','p.MEK','p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK','p.PDPK1','p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']
for x in range(len (genes)):
#     print (x)
    my_list = ['b.CATENIN','cleavedCas','CyclinB','GAPDH','IdU','Ki.67','p.4EBP1','p.Akt.Ser473.', 'p.AKT.Thr308.','p.AMPK','p.BTK','p.CREB','p.ERK','p.FAK','p.GSK3b','p.H3','p.HER2_c','p.JNK','p.MAP2K3','p.MAPKAPK2','p.MEK',   'p.MKK3.MKK6','p.MKK4','p.NFkB','p.p38','p.p53','p.p90RSK','p.PDPK1','p.PLCg2_c','p.RB','p.S6','p.S6K','p.SMAD23','p.SRC','p.STAT1','p.STAT3','p.STAT5']
    if x+1 <len (genes):
#         my_list = genes 
#         print (x,genes[x], genes[x+1])
        my_list.pop(x)
        my_list.pop(x)
#         my_list.pop(x+1)
#         print ( my_list )
        rounds[x] = { "test":[genes[x], genes[x+1]] , "train":my_list}
#         rounds[x] = { "test":[genes[x] ], "train":my_list}
#         y = df[[genes[x], genes[x+1]]]

In [ ]:
rounds[0]["test"]

In [ ]:
# import re
# regex = re.compile(r"\[|\]|<", re.IGNORECASE)
# df.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<','.'))) else col for col in df.columns.values]

In [ ]:
# len(df)

In [ ]:
# type(y)

In [ ]:
# pd.concat([scaled_data,my_dummies_time,my_dummies_treatment])

In [ ]:
# # for m in X.columns :
# #     print (m, len(X[X[m].isna()]))
# with ProgressBar():
# #     pd.DataFrame( data= scaled_data, columns= X.columns )
#     scaled_data_2 = dd.from_pandas( pd.DataFrame( data= scaled_data, columns= X.columns ),npartitions=200)    
 

In [ ]:
# #     print ("adding dummies")
# with ProgressBar():
#     scaled_data_2 = dd.concat([scaled_data_2,my_dummies_treatment,my_dummies_time],axis=1)

In [ ]:
# with ProgressBar():
#     scaled_data_2 = scaled_data.compute()

In [ ]:
# type(scaled_data)

In [ ]:
# type(scaled_data_2)

In [ ]:
# scaled_data_2.head()

In [ ]:
# with ProgressBar():
#     for m in df.columns :
#         print (m, len(df[df[m].isna()]))
# #         y["%s_c"%(m)] =y[m].fillna(y[m].mean() )#, inplace=True )

In [ ]:
for keys in rounds.keys():
    print (keys ,rounds[keys]["train"], rounds[keys]["test"] )

In [ ]:
y = df[['p.STAT3', 'p.STAT5']]

In [ ]:
X = X.compute()

In [ ]:
X.head().compute()

In [ ]:
y = y.compute()

In [ ]:
y

In [ ]:
df[['cleavedCas', 'CyclinB', 'GAPDH', 'IdU', 'Ki.67', 'p.4EBP1', 'p.Akt.Ser473.', 'p.AKT.Thr308.', 'p.AMPK', 'p.BTK', 'p.CREB', 'p.ERK', 'p.FAK', 'p.GSK3b', 'p.H3', 'p.HER2_c', 'p.JNK', 'p.MAP2K3', 'p.MAPKAPK2', 'p.MEK', 'p.MKK3.MKK6', 'p.MKK4', 'p.NFkB', 'p.p38', 'p.p53', 'p.p90RSK', 'p.PDPK1', 'p.PLCg2_c', 'p.RB', 'p.S6', 'p.S6K', 'p.SMAD23', 'p.SRC', 'p.STAT1', 'p.STAT3', 'p.STAT5']].head()

In [ ]:
df['cleavedCas'].head()

In [ ]:
for keys in rounds.keys():
    print (keys ,rounds[keys]["train"], rounds[keys]["test"] )
    X = df[rounds[keys]["train"]]
    y = df[rounds[keys]["test"]]
    

#     X.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]
#     print ("nan")    
#     with ProgressBar():
#         for m in X.columns :
#             print (m, len(X[X[m].isna()]),X[m].max())
#         for m in y.columns :
#             print (m, len(y[y[m].isna()]) ,y[m].max())
#         y["%s_c"%(m)] =y[m].fillna(y[m].mean() )#, inplace=True )
#         y["%s_c"%(m)] =y[m].fillna(y[m].mean() )#, inplace=True )
#     y = y.compute()
    print ("scaling")

    with ProgressBar():
        scaler = StandardScaler()
        scaler.fit(X)
        scaled_data = scaler.transform(X)
    with ProgressBar():
        scaled_data = dd.from_pandas(scaled_data)    
#     print ("adding dummies")
    with ProgressBar():
        scaled_data = dd.concat([scaled_data,my_dummies_treatment,my_dummies_time],axis=1) 
    
        
    print ("split")
    X_train, X_test, y_train, y_test = train_test_split(scaled_data,y,test_size=0.33, random_state=101)
    
    print ("model")
    model = MultiOutputRegressor(RandomForestRegressor(n_jobs=-1,
                                                       random_state=101,
                                                       n_estimators=100,
                                                       criterion='mae',verbose=2)
                                )
#     model = XGBRegressor(n_jobs=-1,random_state=101,n_estimators=100)
    
    print ("train")
    #model.fit(X_train)
#      with ProgressBar():
#     print (X_train.columns)
#     print (y_train.columns)
            
#     X_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in X.columns.values]
#     y_train.columns = [regex.sub("_", col) if any(x in str(col) for x in set(('[', ']', '<'))) else col for col in y.columns.values]
# #     with joblib.parallel_backend('dask'):
#     print (X_train.columns)
#     print (y_train.columns)
    model.fit(X_train, y_train)
   

In [58]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data,y,test_size=0.33, random_state=101)


In [63]:
model = MultiOutputRegressor(RandomForestRegressor(n_jobs=-1,
                                                       random_state=101,
                                                       n_estimators=100,
                                                       criterion='mae',verbose=2))

In [64]:
# with joblib.parallel_backend('dask'):
model.fit(X_train, y_train)
     

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KeyboardInterrupt: 

In [ ]:
subch2 = pd.read_csv("models_templates/subchallenge_2_template_data.csv")

In [ ]:
subch2.columns

In [ ]:
subch2[subch2["treatment"]=="iEGFR"].tail()

In [ ]:
df[df["treatment"]=="iEGFR"].tail()

In [ ]:
df["treatment"].unique().compute()

In [ ]:
subch2["treatment"].unique()

In [ ]:
subch2["time"].unique()

In [ ]:
subch3 = pd.read_csv("models_templates/subchallenge_3_template_data.csv")

In [ ]:
subch3["treatment"].unique()

In [ ]:
subch3["time"].unique()

In [ ]:
subch3.columns

In [ ]:
subch2.cell_line.unique()

In [ ]:
subch3.cell_line.unique()

In [ ]:
df.cell_line.unique().compute()

In [ ]:
subch1 = pd.read_csv("models_templates/subchallenge_1_template_data.csv")

In [ ]:
subch1.columns

In [ ]:
subch3.head()

In [1]:
subCh3=['BT20', 'BT474', 'BT549', 'CAL148', 'CAL51', 'CAL851', 'DU4475',
       'EFM192A', 'EVSAT', 'HBL100', 'HCC1187', 'HCC1395', 'HCC1419',
       'HCC1500', 'HCC1569', 'HCC1599', 'HCC1937', 'HCC1954', 'HCC2185',
       'HCC3153', 'HCC38', 'HCC70', 'HDQP1', 'JIMT1', 'MCF7',
       'MDAMB134VI', 'MDAMB157', 'MDAMB175VII', 'MDAMB361', 'MDAMB415',
       'MDAMB453', 'MFM223', 'MPE600', 'MX1', 'OCUBM', 'T47D', 'UACC812',
       'UACC893', 'ZR7530']

In [2]:
len(subCh3)

39